Librerias

In [36]:
import math
import random
import re
import pandas as pd

Datos

In [37]:
archivo = r"C:\Users\afpue\Documents\GitHub\NLP-Seminar\publicaciones\binaria_limpio.xlsx"
df = pd.read_excel(archivo)
df['Clase'] = df['Clase'].map({'Negativo': 0, 'Positivo': 1})

df['Documento'] = df['Documento_Lematizado']
def limpiar(texto):
    texto = texto.lower()
    return re.findall(r'\b\w+\b', texto)

Diccionarios

In [38]:
palabras_positivas = {"feliz", "alegre", "contento", "maravilloso", "excelente", "genial", "bueno", "positivo", "fantástico", "me encanta",
                      "filantropos", "filantropo"}
palabras_negativas = {"triste", "deprimido", "mal", "horrible", "terrible", "enfermo", "negativo", "odio", "me molesta", "estresado", 
                      "muertes", "muerte", "enfermedad", "dolor", "sufrimiento", "tragedia", "desastre", "crisis", "problema", "conflicto",
                      "infectados", "desgracia"}


Funciones

In [ ]:
def limpiar(texto):

    return texto.lower().split()

# Se estraen las características de cada tweet, esto se puede personalizar como se desee
def extraer_caracteristicas(tweet, palabras_negativas, palabras_positivas):
    tokens = limpiar(tweet)
    x0 = 1  # Bias
    x1 = sum(1 for palabra in tokens if palabra in palabras_negativas)
    x2 = sum(1 for palabra in tokens if palabra in palabras_positivas)
    x3 = len(tokens)
    return [x0, x1, x2, x3]

# Funcion sigmoide
def sigmoid(z):
    return 1 / (1 + math.exp(-z))

# Producto punto entre dos vectores
def dot(w, x):
    return sum(wi * xi for wi, xi in zip(w, x))

# Predicción de probabilidad y^
def predict_proba(w, x):
    return sigmoid(dot(w, x))

# Funcion de perdida
def single_log_loss(p, y):
    p = max(min(p, 1 - 1e-15), 1e-15)
    return - y * math.log(p) - (1 - y) * math.log(1 - p)

# Cálculo del gradiente
def compute_gradient_example(w, xi, yi):
    p = predict_proba(w, xi)
    grad = [(p - yi) * xj for xj in xi]
    return grad

# Entrenamiento de regresión logística con descenso de gradiente estocástico
def train_logistic_regression_sgd(X, y, lr=0.01, epochs=1000):
    w = [0.0 for _ in range(len(X[0]))]  # Inicializar pesos en cero (como en el libro)

    for epoch in range(epochs):
        combined = list(zip(X, y))
        random.shuffle(combined)

        for xi, yi in combined:
            p = predict_proba(w, xi) # Predicción de probabilidad, y^
            loss_i = single_log_loss(p, yi) # Pérdida de y^ respecto a y
            grad = compute_gradient_example(w, xi, yi) # Cálculo del gradiente
            w = [wj - lr * gj for wj, gj in zip(w, grad)] # Actualización de pesos

        # Promedio de pérdida de la época, esto solo es informativo para saber como va el entrenamiento
        if epoch % 100 == 0 or epoch == epochs - 1:
            print(f"Época {epoch}: pérdida último ejemplo = {loss_i:.4f}")

    return w

# Predicción binaria, se puede ajustar el umbral segun la necesidad
def predict(w, x, threshold=0.5):
    return 1 if predict_proba(w, x) >= threshold else 0


In [ ]:
# seleccion del dato de prueba
tweet_test = df.iloc[-1]['Documento']
y_test = df.iloc[-1]['Clase']
x_test = extraer_caracteristicas(tweet_test, palabras_negativas, palabras_positivas)

# Dejo el resto como entrenamiento
df_train = df.iloc[:-1]
X_train = [extraer_caracteristicas(texto, palabras_negativas, palabras_positivas) for texto in df_train['Documento']]
y_train = df_train['Clase'].tolist()


In [ ]:
# Entrenamiento del modelo
w = train_logistic_regression_sgd(X_train, y_train, lr=0.01, epochs=1000)

# Predicción final
proba = predict_proba(w, x_test)
clase_predicha = predict(w, x_test)

# Resultado final
print("\n--- Resultado final ---")
print("Tweet:", tweet_test)
print("Clase real:", y_test)
print("Clase predicha:", clase_predicha)
print(f"Probabilidad estimada de clase positiva: {proba:.4f}")


Época 0: pérdida último ejemplo = 0.3939
Época 100: pérdida último ejemplo = 1.1934
Época 200: pérdida último ejemplo = 0.1727
Época 300: pérdida último ejemplo = 0.3284
Época 400: pérdida último ejemplo = 0.2938
Época 500: pérdida último ejemplo = 0.5512
Época 600: pérdida último ejemplo = 2.8434
Época 700: pérdida último ejemplo = 0.0081
Época 800: pérdida último ejemplo = 0.4690
Época 900: pérdida último ejemplo = 1.4548
Época 999: pérdida último ejemplo = 2.9443

--- Resultado final ---
Tweet: el mundo yo parecer más amable , más humano , menos raro .
Clase real: 1
Clase predicha: 1
Probabilidad estimada de clase positiva: 0.9828


In [42]:
w

[1.2796314493293743, -5.444717875507482, 0.0, 0.21270798947939554]